Created by Florent Poux. Licence CC

*   To reuse in your project, please cite the article.
*   Have fun with this notebook that you can very simply run (ctrl+Enter) !
*   The first time thought, it will ask you to get a key for it to be able to acces your Google drive folders if you want to work all remotely.
*   Simply accept, and then change the "10-MEDIUM/DATA/Point Cloud Sample/" by the folder path containing your data

Enjoy!

# Step 1: Setting up the environment

In [1]:
#http://www.open3d.org/docs/release/index.html
# !pip install open3d

# Step 2: Load and prepare the data

In [2]:
#libraries used
import numpy as np
import os
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import time

In [4]:
#create paths and load data
# Where to save the figures
PROJECT_ROOT_DIR = ".."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "png_images")
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "dataset")
XYZ_point_PATH = os.path.join(DATA_PATH, "xyz")
PCD_point_PATH = os.path.join(DATA_PATH, "pcd")
file_output_dir =  os.path.join(PROJECT_ROOT_DIR, "output_dir/surface_fitting_result")
os.makedirs(IMAGES_PATH, exist_ok=True)
os.makedirs(file_output_dir, exist_ok=True)

## function for automatically save the diagram/graph into the folder 
def save_fig_plt(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")



point_cloud= np.loadtxt(os.path.join(XYZ_point_PATH, "30_seg.xyz"),skiprows=1)

In [5]:
pcd = o3d.io.read_point_cloud(os.path.join(PCD_point_PATH, "30_seg.pcd")) #H1_seg
hull_mesh = pcd.compute_convex_hull()
# hull_mesh
o3d.visualization.draw_geometries([hull_mesh[0]])

In [6]:
t0 = time.monotonic()

In [7]:
#Format to open3d usable objects
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
# pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6].astype(np.float)/255.0)  
# pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

In [8]:
# !pip install pyvista
# !pip install ipyvtklink

In [9]:
# import pyvista as pv
# from pyvista import examples

# # download an example and display it using physically based rendering.
# mesh = examples.download_lucy()
# mesh.plot(color='lightgrey', pbr=True, metallic=0.2,
#           # jupyter_backend='pythreejs')

In [10]:
# ---- using numpy to print all the points -------
points_np_array = np.asarray(pcd.points)
print("the points converted to numpy are: \n")
print(points_np_array)
#------extract x,y,z value of the points-----
# ------------standarization-----------
# bbdiag = float(np.linalg.norm(points_np_array.max(0) - points_np_array.min(0), 2))
# points_np_array = (points_np_array - points_np_array.mean(0))/bbdiag
# print("the points are: \n")
# print(points_np_array)

scale = 1
x = points_np_array[:,0]* scale ## from m to mm
y = points_np_array[:,1]*scale  ## from m to mm
points_np_array[:,2] = points_np_array[:,2] - np.amin(points_np_array[:,2]) 
z = points_np_array[:,2]*scale  ## from m to mm
print("the z value extracted:" )
print (z)

# --------mean height-------------------
z_mean = np.mean(z)
print("Z mean: " + str(z_mean))

z_min = np.amin(z)
z_max = np.amax(z)
print ("Z minimum: "+ str(z_min))
print ("Z max: "+ str(z_max))

the points converted to numpy are: 

[[-0.090195    0.133964   -0.22784901]
 [-0.09033533  0.13413933 -0.227835  ]
 [-0.090437    0.13429749 -0.227854  ]
 ...
 [-0.063091    0.160614   -0.224999  ]
 [-0.062908    0.160607   -0.224997  ]
 [-0.063478    0.16084699 -0.224995  ]]
the z value extracted:
[4.990e-06 1.900e-05 0.000e+00 ... 2.855e-03 2.857e-03 2.859e-03]
Z mean: 0.0015219748623726958
Z minimum: 0.0
Z max: 0.002912090000000006


In [11]:
import numpy as np
import pyvista as pv


cloud = pv.PolyData(point_cloud)
cloud['point_color'] = cloud.points[:, 2]

pv.plot(cloud,  cmap='jet', show_bounds=False)
# cloud.plot(jupyter_backend='pythreejs',cmap='jet', show_bounds=True)
# cloud.plot(jupyter_backend='ipygany', show_scalar_bar=True)
# cloud.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [35]:
point_cloud_boonlean = np.asarray(pcd.points)
point_cloud_boonlean[:,0] = points_np_array[:,0]
point_cloud_boonlean[:,1] = points_np_array[:,1]
point_cloud_boonlean[:,2] = (z_max) - points_np_array[:,2] 
cloud_boonlean = pv.PolyData(point_cloud_boonlean)
cloud_boonlean['point_color_boolean'] = cloud_boonlean.points[:, 2]
# pv.plot(cloud_boonlean,  cmap='jet', show_bounds=False)
pv.plot(cloud_boonlean, scalars='point_color_boolean', cmap='jet', show_bounds=False)

# create many spheres from the point cloud
sphere = pv.Sphere(radius=0.00001, phi_resolution=10, theta_resolution=10)
pc_boonlean = cloud_boonlean.glyph(scale=False, geom=sphere)

# pc.plot(cmap='jet') #ipyvtklink,pythreejs
# pv.plot(cloud_boonlean, scalars='point_color', cmap='jet', show_bounds=False)

volume_boolean = pc_boonlean.delaunay_3d(alpha=0.001)
shell_5_boolean = volume_boolean.extract_geometry()
# shell.plot(jupyter_backend='pythreejs',cmap='jet') # ipvtklink is very slow
shell_5_boolean.plot(cmap='jet')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [13]:
# create many spheres from the point cloud
sphere = pv.Sphere(radius=0.001, phi_resolution=1, theta_resolution=1)
pc = cloud.glyph(scale=False, geom=sphere)

# pc.plot(cmap='jet',jupyter_backend='pythreejs') #ipyvtklink,pythreejs
pv.plot(cloud, scalars='point_color', cmap='jet', show_bounds=False)
# pv.plot(cloud, scalars='point_color', cmap='jet', show_bounds=True)
# pc.plot(cmap='jet') 

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [14]:
volume = pc.delaunay_3d(alpha=5)
shell_5 = volume.extract_geometry()
# shell.plot(jupyter_backend='pythreejs',cmap='jet') # ipvtklink is very slow
shell_5.plot(cmap='jet') 

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [15]:
volume = pc.delaunay_3d(alpha=0.1)
shell = volume.extract_geometry()
# shell.plot(jupyter_backend='pythreejs',cmap='jet') # ipvtklink is very slow
# shell.plot(cmap='jet') 

In [16]:
volume = pc.delaunay_3d(alpha=1)
shell = volume.extract_geometry()
# shell.plot(jupyter_backend='pythreejs',cmap='jet') # ipvtklink is very slow
# shell.plot(cmap='jet') 

In [17]:
volume = cloud.delaunay_3d(alpha=10)
shell = volume.extract_geometry()
# shell.plot(jupyter_backend='pythreejs',cmap='jet') # ipvtklink is very slow,
shell.plot(cmap='jet') 

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [18]:
volume = pc.delaunay_3d(alpha=100)
shell = volume.extract_geometry()
# shell.plot(jupyter_backend='pythreejs',cmap='jet') # ipvtklink is very slow
shell.plot(cmap='jet') 

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [19]:
volume = pc.delaunay_3d(alpha=0.01)
shell = volume.extract_geometry()
# shell.plot(jupyter_backend='pythreejs',cmap='jet') # ipvtklink is very slow
# shell.plot(cmap='jet') 

In [20]:
t1 = time.monotonic()
print (t1 - t0)

3.985000000000582


In [21]:
p = pv.Plotter(window_size=np.array([1024, 768])*2)

# Add all the data we want to see
p.add_mesh(shell_5, cmap="nipy_spectral", opacity=0.15)

# Add a title
p.add_text("converted mesh visualization")

# A nice perspective
# p.camera_position = [(544065.5831913119, 3924518.576093113, 24324.3096344195),
#                      (597885.1732914157, 3982998.0900773173, -12587.537450058662),
#                      (0.33162714740718435, 0.26609487244915314, 0.9051060456978746)]
p.show( screenshot="mesh.png")

ViewInteractiveWidget(height=1536, layout=Layout(height='auto', width='100%'), width=2048)

In [22]:
# mesh points
# vertices = np.array([[-16, -16, 3],
#                      [16, -16, 3],
#                      [16, 16, 3],
#                      [-16, 16, 3]])

# # mesh faces
# # faces = np.hstack([[-16,-16,16,16]])    

# surf = pv.PolyData(vertices, None)

# # plot each face with a different color
# surf.plot(scalars=np.arange(1), cpos=[-1, 1, 0.5])


In [23]:
cyl = pv.Cylinder()
arrow = pv.Arrow()
sphere = pv.Sphere()
plane = pv.Plane(center=(0, 0, 0), direction=(0, 0, 1), i_size=16, j_size=16, i_resolution=1, j_resolution=1)
line = pv.Line()
box = pv.Box(bounds=(- 16.0, 16.0, -16.0, 16.0, 0, 3))
cone = pv.Cone()
poly = pv.Polygon()
disc = pv.Disc()

In [24]:
p = pv.Plotter(shape=(1,4))
# Top row
p.subplot(0,0)
p.add_mesh(plane, color="tan", show_edges=True)
p.subplot(0,1)
p.add_mesh(box, color="tan", show_edges=True)
# Bottom row
p.subplot(0,2)
p.add_mesh(poly, color="tan", show_edges=True)
p.subplot(0,3)
p.add_mesh(shell_5, cmap="nipy_spectral", opacity=0.15)

# Render all of them
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [25]:
## boonlean operation
result = (shell_5.triangulate()).boolean_difference(box.triangulate())
pl = pv.Plotter()
# _ = pl.add_mesh(box, color='r', style='wireframe', line_width=3)
# _ = pl.add_mesh(shell_5, color='b', style='wireframe', line_width=3)
_ = pl.add_mesh(result, color='tan')
# pl.camera_position = 'xz'
pl.show()

ERROR:root:No points to subdivide
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on


ValueError: Empty meshes cannot be plotted. Input mesh has zero points.

# Step 3: Choose a meshing strategy

Now we are ready to start the surface reconstruction process by meshing the pcd point cloud. I will give my favorite way to efficiently obtain results, but before we dive in, some condensed details ar necessary to grasp the underlying processes. I will limit myself to two meshing strategies. See article

# Step 4: Process the data
## Strategy 1: BPA

In [ ]:
#radius determination
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [ ]:
#computing the mehs
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

In [ ]:
#decimating the mesh
dec_mesh = mesh.simplify_quadric_decimation(100000)

*Optional ---*

In [ ]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

## Strategy 2: Poisson' reconstruction

In [ ]:
#computing the mesh
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [ ]:
#cropping
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

# Step 5: Export and visualize

In [ ]:
#export
o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", dec_mesh)
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", p_mesh_crop)

In [ ]:
#function creation
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
#execution of function
my_lods = lod_mesh_export(bpa_mesh, [100000,50000,10000,1000,100], ".ply", output_path)

In [ ]:
#execution of function
my_lods2 = lod_mesh_export(bpa_mesh, [8000,800,300], ".ply", output_path)